In [3]:
import numpy as np
import pandas as pd
import json

df = pd.read_csv('../data/tt_asignaciones_samp.csv')

In [4]:
with open('../utils.json') as f:
    dd = json.load(f)

def tweak_asignaciones(df):
    return (df
     .drop(columns=['asa_beca']) # columns contains 0 only; uninformative
     .rename(columns={'nombre_institucion':'ies_nombre',
                      'cod_prov_reside':'cod_provincia_reside',
                      'provincia':'provincia_ies', 'canton':'canton_ies', 
                      'parroquia':'parroquia_ies'})
     .assign(acepta_cupo=lambda df_: (df_
                                      .acepta_cupo
                                      .replace(2, np.nan) # replacing meaningless 2 with np.nan
                                      .astype('uint8', errors='ignore')),
             etnia=lambda df_: (df_.etnia
                                .str.lower()
                                .replace(regex=dd['etnias'])
                                .str.title()
                                .replace(regex={'Ind...ena':'Indígena'})
                                .astype('category')),
             parroquia_ies=lambda df_: (df_
                                        .parroquia_ies
                                        .str.title()
                                        .replace({'Abraham Calazacã\x93N':'Abraham Calazacón',
                                                  'Azogues' : 'Azogues, Cabecera Cantonal',
                                                  'Bahã\x8dA De Carã\x81Quez': 'Bahía De Caráquez',
                                                  'Baã\x91Os De Agua Santa' : 'Baños De Agua Santa',
                                                  'Carcelã\x89N' : 'Carcelén', 'Caã\x91Ar' : 'Cañar',
                                                  'Cariamanga' : 'Cariamanga, Cabecera Cantonal',
                                                  'Cayambe' : 'Cayambe, Cabecera Cantonal',
                                                  'El Ã\x81Ngel' : 'El Ángel', 'Iã\x91Aquito' : 'Iñaquito',
                                                  'La Aurora (Satã\x89Lite)' : 'La Aurora (Satélite)',
                                                  'La Concepciã\x93N' : 'La Concepción',
                                                  'Portoviejo' : 'Portoviejo, Cabecera Cantonal Y Capital Provincial',
                                                  'Quevedo' : 'Quevedo, Cabecera Cantonal',
                                                  'Santa Elena' : 'Santa Elena, Cabecera Cantonal Y Capital Provincial'})
                                        .replace(['Atocha \x96 Ficoa', 'Atocha Â\x80\x93 Ficoa'], 'Atocha Ficoa')
                                        .replace(['Sangolqui, Cabecera Cantonal', 'Sangolquã\x8d'], 'Sangolquí')
                                        .replace(['Santo Domingo De Los Colorados','Santo Domingo De Los Colorados, Cabecera Cantonal'], 
                                                 'Santo Domingo De Los Tsachilas, Cabecera Cantonal Y Capital Provincial')
                                        .replace(['Tulcan, Cabecera Cantonal', 'Tulcán', 'Tulcã\x81N'],
                                                 'Tulcán, Cabecera Cantonal Y Capital Provincial')
                                        .astype('category')),
             **{c: lambda df_, c=c: df_[c].astype('uint8', errors='ignore') for c in ['periodo', 
                                                                                      'pos_prioridad', 
                                                                                      'cod_provincia_reside']},
             **{c: lambda df_, c=c: df_[c].astype('uint16', errors='ignore') for c in ['car_id', 
                                                                                       'pos_nota', 'ies_id',
                                                                                       'cod_canton_reside']},
             **{c: lambda df_, c=c: df_[c].astype('uint32', errors='ignore') for c in ['ofa_id', 'pos_id', 
                                                                                       'cod_parroquia_reside']},
             **{c: lambda df_, c=c: df_[c].str.title().astype('category') for c in ['ies_nombre', 'provincia_ies', 
                                                                                    'canton_ies', 'carrera', 'area', 
                                                                                    'modalidad']},
             genero=lambda df_: df_.genero.replace({'MASCULINO':1, 'FEMENINO':0}).astype('uint8', errors='ignore'), # 1: Masculino ; 0: Femenino
             discapacidad=lambda df_: df_.discapacidad.replace({'NO':0, 'SI':1}).astype('uint8', errors='ignore'),
             universidad=lambda df_: df_.ies_nombre.str.contains('Universidad'),
             in_capital=lambda df_: df_.parroquia_ies.str.contains('Capital')
             )
    )

adf = tweak_asignaciones(df)

In [5]:
adf.head().T

,0,1,2,3,4
genero,1.0,1.0,1.0,0.0,0.0
etnia,Mestizo,Indígena,Mestizo,Mestizo,Mestizo
cod_parroquia_reside,10113.0,60104.0,170256.0,220350.0,170164.0
cod_canton_reside,101.0,601.0,1702.0,2203.0,1701.0
cod_provincia_reside,1.0,6.0,17.0,22.0,17.0
pos_id,16516144,17825859,25735501,28452649,24917400
pos_nota,784,774,759,841,760
pos_prioridad,1,1,1,1,2
ies_id,48,58,804,877,46
ies_nombre,Universidad De Cuenca,Universidad Estatal De Bolivar,Universidad Regional Amazonica Ikiam,Instituto Tecnológico Superior General Eloy Al...,Universidad Central Del Ecuador


### Univariate Data Analysis